This notebook queries data from our twitter-topic-sentiment Flask api and displays some results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import json

### Request to our FLASK API 

In [ ]:
REST_API_URL = "localhost/" #path to the server exposing your FLASK API
request_url = REST_API_URL + "sentiment"

In [ ]:
hashtag = '#twitter'
request_parameters = {'hashtag': hashtag, 'nb_days': 7, 'nb_tweets': 70,
                      'get_topic_words': True,'n_topics': 10,'n_words_per_topic': 10,
                      'lda_passes': 2,'return_tweets': False,'language':'en'}
response = requests.get(request_url,params=request_parameters)
response_dict = json.loads(response.json())

In [ ]:
# tweets_sentiments_df = pd.from_json(response_dict["top_topics"])


### Sentiment visualization

In [ ]:
tweets_sentiments_df = pd.read_json(api_results["sentiments_json"])
tweets_sentiments_df.created_at = pd.to_datetime(tweets_sentiments_df.created_at).dt.date

In [ ]:
plt.figure(figsize=(20,5))
sns.barplot(x="created_at",y="pos_sentiment",color = "green",label="positive sentiment",capsize=.2,data=tweets_sentiments_df)
ax = sns.barplot(x="created_at",y="neg_sentiment",color = "red",label="negative sentiment",capsize=.2,data=tweets_sentiments_df)

ax.set_ylabel("sentiment")
plt.xticks(rotation=10)
plt.title("Sentiment polarity (over {} tweets) during last week for hastag {} : ".format(len(tweets_sentiments_df),hashtag))
plt.legend()
plt.show()

### Topic visualization

In [ ]:
top_topics_df = pd.read_json(api_results["top_topics_json"])

In [ ]:
### Word Cloud
topic_num = 0
topic_to_plot = top_topics_df.loc[top_topics_df.topics==topic_num]
frequencies_dict = {topic_to_plot.iloc[i]["words"]:topic_to_plot.iloc[i]["proba"] for i in range(len(topic_to_plot))}

wordcloud = WordCloud()
wordcloud.generate_from_frequencies(frequencies_dict)

plt.figure(figsize=(20,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud()
wordcloud.generate_from_frequencies(frequencies_dict)

plt.figure(figsize=(20,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()